### Lab: Celery
* create a Celery task named __`write_file()`__ which takes two arguments, a filename and a string, and write the string to the file
* create a second task named __`read_file()`__ which reads from a file
  * if the file does not exist, it waits until the file exists (perhaps have it poll once a second)
* test your tasks


In [1]:
%%file data/celery_examples/file_tasks.py
import os
import time

from celery import Celery

app = Celery('tasks', backend='rpc://', broker='amqp://guest@localhost')

@app.task
def write_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

@app.task
def read_file(filename):
    while not os.path.exists(filename):
        time.sleep(1)
    with open(filename) as f:
        data = f.read()
    return data

Overwriting data/celery_examples/file_tasks.py


In [2]:
import sys
sys.path.append('data')

In [3]:
from celery_examples.file_tasks import write_file, read_file

filename = input('Filename? ')
content = input('Content? ')

write_file.delay(filename, content)
result = read_file.delay(filename)
print(result.get())


Filename? test_abc.txt
Content? This is my test file
This is my test file


In [5]:
cat data/test_abc.txt

This is my test file

In [7]:
filename = 'test_xyz.txt'

In [8]:
import threading
def reader():
    result = read_file.delay(filename)
    print(result.get())
thread = threading.Thread(target=reader)
thread.start()

In [9]:
write_file.delay(filename, 'Here is a file')

<AsyncResult: c1ac635d-e296-47e9-bf03-b3d999d88213>

Here is a file


In [10]:
thread.isAlive()

False